In [26]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import pandas as pd
import re
import shutil
from sklearn.model_selection import StratifiedKFold
import random
import os
import numpy as np
import torch
import matplotlib.pyplot as plt

In [3]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
HOME_FOLDER = '/media/raid/astamoulakatos/nsea_frame_sequences/centre_Ch2/'

noOfFiles = 0
noOfDir = 0

In [ ]:
for base, dirs, files in os.walk(HOME_FOLDER):
    print('Looking in : ',base)
    for directories in dirs:
        noOfDir += 1
    for Files in files:
        noOfFiles += 1

In [ ]:
print('Number of files',noOfFiles)
print('Number of Directories',noOfDir)
print('Total:',(noOfDir + noOfFiles))

In [ ]:
number_of_frames = []
event_path = []
for d in os.listdir(HOME_FOLDER):
    label_path = os.path.join(HOME_FOLDER,d)
    for l in os.listdir(label_path):
        file_path = os.path.join(label_path,l)
        #print(len(os.listdir(file_path)), file_path)
        number = len(os.listdir(file_path))
        number_of_frames.append(number) 
#         if number > 400:
#             shutil.rmtree(file_path)
#         else:
#             number_of_frames.append(number)                          
        event_path.append(file_path)

In [ ]:
df = pd.DataFrame(columns = ['event_path', 'number_of_frames'])

In [ ]:
df.event_path = event_path
df.number_of_frames = number_of_frames

In [ ]:
df

In [ ]:
df.number_of_frames.max()

In [ ]:
df.number_of_frames.min()

In [ ]:
df.number_of_frames.mean()

In [ ]:
df['exposure'] = 0
df['burial'] = 0
df['field_joint'] = 0
df['anode'] = 0
df['free_span'] = 0

In [ ]:
labels = []
for i in range(len(df)):
    m = re.search('Ch2/(.+?)/S', str(df.event_path[i]))
    #m = re.search('egs/(.+?)/S', str(df.event_path[i]))
    boom = m.group(1)
    if boom == 'exp_and':
        df.exposure[i] = 1
        df.anode[i] = 1
    if boom == 'exp':
        df.exposure[i] = 1
    if boom == 'bur':
        df.burial[i] = 1
    if boom == 'exp_fs':
        df.exposure[i] = 1
        df.free_span[i] = 1
    if boom == 'exp_fj':
        df.exposure[i] = 1
        df.field_joint[i] = 1
    labels.append(boom)

In [ ]:
df['label'] = labels

In [ ]:
df1 = df[['event_path','exposure','burial','field_joint','anode','free_span']]

In [ ]:
fig1, ax1 = plt.subplots()
df1.iloc[:,1:].sum(axis=0).plot.pie(autopct='%1.1f%%',shadow=True, startangle=90,ax=ax1)
ax1.axis("equal")
plt.show()

# events

In [ ]:
df1.iloc[:,1:].sum(axis=0)

# frames

In [ ]:
df.groupby('label')['number_of_frames'].sum()

# exclude events with less than 50 frames (2 secs)

In [ ]:
df.number_of_frames.min()

In [ ]:
df = df[df.number_of_frames>=50]
df = df[df.number_of_frames<=400]

In [ ]:
df

In [ ]:
df.label.unique()

# less exposure

In [ ]:
df_exp = df[df.label=='exp']

In [ ]:
df_exp = df_exp.sample(frac=0.3)

In [ ]:
df_exp

In [ ]:
df_exp.number_of_frames.mean()

In [ ]:
df_new = df[df.label!='exp']

In [ ]:
df_final = pd.concat([df_new, df_exp])

In [ ]:
df_final.reset_index(drop=True)

In [ ]:
df_final.label.value_counts()

# frame sequences

In [ ]:
df_final.groupby('label')['number_of_frames'].sum()

# distribution of labels in events, not in sequences, not in frames

In [ ]:
df1 = df_final[['event_path','exposure','burial','field_joint','anode','free_span']]

In [ ]:
fig1, ax1 = plt.subplots()
df1.iloc[:,1:].sum(axis=0).plot.pie(autopct='%1.1f%%',shadow=True, startangle=90,ax=ax1)
ax1.axis("equal")
plt.show()

In [ ]:
df_final.to_csv('../important_csvs/more_balanced_dataset/events_with_number_of_frames_less_exp.csv', index=False)

In [ ]:
df_final['number_of_frames'].hist(by=df.label)
plt.show()

# stratification

In [ ]:
df = df_final

In [ ]:
df.reset_index(drop=True)

In [ ]:
SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [ ]:
skf = StratifiedKFold(n_splits=5)

In [ ]:
df['fold'] = 0

In [ ]:
g = df.groupby('label')

In [ ]:
i = g['number_of_frames'].quantile([0.05, 0.25, 0.5, 0.9]).unstack()
j = g['number_of_frames'].agg(['min', 'max'])

In [ ]:
pd.concat([i, j], 1)

In [ ]:
values = [400, 400, 99, 103, 276]

In [ ]:
i.T.plot(subplots=True)
plt.show()

In [ ]:
df['stratify_group'] = 'yo'

In [ ]:
labels = df.label.unique()

In [ ]:
for l, v in zip(labels, values):
    df.loc[df.label==l, 'stratify_group'] = np.char.add(
        df.loc[df.label==l, 'label'].values.astype(str),
        df.loc[df.label==l, 'number_of_frames'].apply(lambda x: f'_{int(x // v)}').values.astype(str)
    )

In [ ]:
df

In [ ]:
for fold_number, (train_index, val_index) in enumerate(skf.split(X=df['event_path'], y=df['stratify_group'])):
    df.loc[df.iloc[val_index].index, 'fold'] = fold_number

In [ ]:
df.to_csv('../important_csvs/more_balanced_dataset/more_balanced_stratified.csv', index=False)

In [ ]:
df.fold.value_counts()

In [ ]:
df.stratify_group.value_counts()

In [10]:
df = pd.read_csv('../important_csvs/more_balanced_dataset/more_balanced_stratified.csv')

In [5]:
df = df.sample(frac=0.5)

In [17]:
df = df.reset_index(drop=True)

In [18]:
df

,event_path,number_of_frames,exposure,burial,field_joint,anode,free_span,label,fold,stratify_group
0,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
1,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
2,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,102,0,1,0,0,0,bur,0,bur_0
3,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
4,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,175,0,1,0,0,0,bur,0,bur_0
...,...,...,...,...,...,...,...,...,...,...
1095,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1096,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1097,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1098,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,358,1,0,0,0,0,exp,4,exp_1


In [19]:
df.fold.value_counts()

0    224
1    222
2    220
4    218
3    216
Name: fold, dtype: int64

In [20]:
df.label.value_counts()

bur        315
exp_fj     257
exp_fs     239
exp        236
exp_and     53
Name: label, dtype: int64

In [ ]:
for i in range(len(df)):
    if (df.label[i] != 'exp'):
        df.exposure[i] = 0

In [ ]:
df

In [ ]:
df.to_csv('../important_csvs/more_balanced_dataset/small_set_multi_class.csv', index=False)

In [21]:
df.number_of_frames.min()

100

In [22]:
df.number_of_frames.max()

400

In [15]:
df = df[df.number_of_frames>=100]

In [16]:
df

,event_path,number_of_frames,exposure,burial,field_joint,anode,free_span,label,fold,stratify_group
0,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
1,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
2,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,102,0,1,0,0,0,bur,0,bur_0
3,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,400,0,1,0,0,0,bur,0,bur_1
4,/media/scratch/astamoulakatos/centre_Ch2/bur/S...,175,0,1,0,0,0,bur,0,bur_0
...,...,...,...,...,...,...,...,...,...,...
1432,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1433,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1434,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
1435,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,358,1,0,0,0,0,exp,4,exp_1


In [23]:
df.label.value_counts()

bur        315
exp_fj     257
exp_fs     239
exp        236
exp_and     53
Name: label, dtype: int64

# less exposure

In [25]:
df_exp = df[df.label=='exp']

In [24]:
df_exp = df_exp.sample(frac=0.25)

NameError: name 'df_exp' is not defined

In [17]:
df_exp

,event_path,number_of_frames,exposure,burial,field_joint,anode,free_span,label,fold,stratify_group
540,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
460,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
94,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,1,exp_1
203,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,2,exp_1
674,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
332,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,3,exp_1
599,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
372,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
144,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
180,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,1,exp_1


In [18]:
df_exp.number_of_frames.mean()

390.5416666666667

In [19]:
df_new = df[df.label!='exp']

In [20]:
df_final = pd.concat([df_new, df_exp])

In [21]:
df_final.reset_index(drop=True)

,event_path,number_of_frames,exposure,burial,field_joint,anode,free_span,label,fold,stratify_group
0,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,107,1,0,1,0,0,exp_fj,4,exp_fj_1
1,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,152,1,0,1,0,0,exp_fj,3,exp_fj_1
2,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,117,1,0,1,0,0,exp_fj,2,exp_fj_1
3,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,161,1,0,1,0,0,exp_fj,0,exp_fj_1
4,/media/scratch/astamoulakatos/centre_Ch2/exp_f...,203,1,0,0,0,1,exp_fs,4,exp_fs_0
...,...,...,...,...,...,...,...,...,...,...
435,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,265,1,0,0,0,0,exp,4,exp_0
436,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,0,exp_1
437,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,4,exp_1
438,/media/scratch/astamoulakatos/centre_Ch2/exp/S...,400,1,0,0,0,0,exp,1,exp_1


In [22]:
df_final.label.value_counts()

bur        149
exp_fj     123
exp_fs     120
exp         24
exp_and     24
Name: label, dtype: int64